In [1]:
import pandas as pd
import numpy as np
import lequi
from netCDF4 import Dataset
import mean_net
import seaborn as sns
from shapely.geometry import Point, Polygon

In [2]:
ISOTOPO = '15N'
n = 10
names_bio = ['bio1','bio2','bio3','bio4','bio5','bio6','bio7','bio8','bio9','bio10','bio11','bio12','bio13','bio14','bio15','bio16','bio17','bio18','bio19']

In [3]:
def call_var(name, n):
    dataset = Dataset('C:/Users/bmati/Desktop/proyecto296/training/data/biocliamtic_CR2_1979_2016/'+name+'.nc', mode="r")
    z = dataset.variables[name][:]
    grid_lat = dataset.variables['lat'][:]
    grid_lon = dataset.variables['lon'][:]
    dataset.close()
    return z[-n:], grid_lat, grid_lon

def significance(pvalue):
    if pvalue < 0.001:
        return ' ***'
    elif pvalue < 0.01:
        return ' **'
    elif pvalue < 0.05:
        return ' *'
    else:
        return ''

In [4]:
df_poligon = pd.read_csv('data/cuencas_poligonos.csv')


In [5]:
def cluster(df_poligon, cuenca, df, zona):
    poli = list(eval(df_poligon[cuenca][0]))
    poligon = Polygon(poli)
    list_value = []
    for i in range(0, len(df['Latitude'])):

        point = Point((df['Longitude'][i],df['Latitude'][i]))
        if poligon.contains(point):
            list_value.append(1)   
        else:
            list_value.append(0)

  
    df[cuenca] = list_value
        
    return df

# cargar datos isotopicos y generar df con variables bioclimaticas

Para delta 13 C en plantas eliminar C4
Para delta 13 C en plantas usar acidificadas

In [6]:
data = pd.read_csv("data/suelosAll.csv")
lon = np.array(data["lon"])
lat = np.array(data["lat"])
iso = np.array(data[ISOTOPO])
lon, lat, iso = lequi.csv(lon),lequi.csv(lat),lequi.csv(iso)
       
bio_layers = {}
for name in names_bio:
    layer, grid_lat, grid_lon = call_var(name, n)
    col = mean_net.returnDat(lat,lon,grid_lat,grid_lon,np.nanmean(layer, axis=0))
    bio_layers[name] = col

bio_layers['Latitude'] = lat
bio_layers['Longitude'] = lon
bio_layers[ISOTOPO] = iso

df = pd.DataFrame(bio_layers)

<ipython-input-3-e3a3b27063c8>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  z = dataset.variables[name][:]
<ipython-input-3-e3a3b27063c8>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  grid_lat = dataset.variables['lat'][:]
<ipython-input-3-e3a3b27063c8>:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence

In [7]:
#Solo para plantas en 13C

df = df[df['15N']> -5]
df = df[df['15N']< 15]

df = df.reset_index()

df

,index,bio1,bio2,bio3,bio4,bio5,bio6,bio7,bio8,bio9,...,bio13,bio14,bio15,bio16,bio17,bio18,bio19,Latitude,Longitude,15N
0,0,16.680066,19.940985,73.603150,222.898539,30.221051,3.088484,27.132568,17.506520,18.646753,...,4.613000,0.0,60.611061,4.751000,0.0,0.000,3.476000,-22.85000,-69.2600,13.55
1,1,13.945570,16.422479,66.199236,302.465765,26.373077,1.530497,24.842580,13.844643,15.541803,...,8.833000,0.0,101.518006,9.362000,0.0,2.718,3.804000,-23.52000,-67.9700,2.92
2,2,17.292080,7.011714,49.380242,201.289604,25.103611,10.860042,14.243569,18.758515,19.791519,...,3.282000,0.0,34.821223,3.308000,0.0,2.485,1.177000,-23.50000,-70.5400,12.88
3,3,8.004621,13.752827,67.942430,172.048517,18.357537,-1.905314,20.262851,9.505382,7.752109,...,50.704000,0.0,190.829799,80.729999,0.0,59.217,1.203000,-18.36000,-69.5600,6.38
4,4,19.663313,6.021628,46.403303,214.393737,27.463324,14.379219,13.084105,21.727257,22.187209,...,0.004000,0.0,0.110187,0.004000,0.0,0.000,0.004000,-18.40000,-70.3400,-1.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,135,15.688830,13.730183,58.347884,338.546730,28.680324,5.104891,23.575432,11.618111,18.571219,...,73.083000,0.0,183.600223,116.499001,0.0,2.352,106.626000,-30.98022,-71.0612,4.16
134,136,15.272948,14.016008,59.113067,336.015721,28.228734,4.478181,23.750553,11.251113,18.414650,...,67.519999,0.0,184.767328,101.455999,0.0,2.761,82.951999,-30.71162,-71.1770,5.09
135,137,15.272948,14.016008,59.113067,336.015721,28.228734,4.478181,23.750553,11.251113,18.414650,...,67.519999,0.0,184.767328,101.455999,0.0,2.761,82.951999,-30.71162,-71.1770,6.32
136,138,15.211451,13.884745,59.017968,336.079665,28.098760,4.505688,23.593072,11.148939,18.697681,...,69.826999,0.0,181.998907,107.880999,0.0,2.588,96.348999,-30.88676,-71.1931,3.06


In [8]:
cuencas_norte = ['Atacama', 'Arica']
cuencas_sur = ['La_Serena', 'Santiago', 'Maule']


for cuenca in cuencas_norte:
    df = cluster(df_poligon, cuenca, df, 'Norte')
    
    
for cuenca in cuencas_sur:
    df = cluster(df_poligon, cuenca, df, 'sur')
    
df

,index,bio1,bio2,bio3,bio4,bio5,bio6,bio7,bio8,bio9,...,bio18,bio19,Latitude,Longitude,15N,Atacama,Arica,La_Serena,Santiago,Maule
0,0,16.680066,19.940985,73.603150,222.898539,30.221051,3.088484,27.132568,17.506520,18.646753,...,0.000,3.476000,-22.85000,-69.2600,13.55,1,0,0,0,0
1,1,13.945570,16.422479,66.199236,302.465765,26.373077,1.530497,24.842580,13.844643,15.541803,...,2.718,3.804000,-23.52000,-67.9700,2.92,1,0,0,0,0
2,2,17.292080,7.011714,49.380242,201.289604,25.103611,10.860042,14.243569,18.758515,19.791519,...,2.485,1.177000,-23.50000,-70.5400,12.88,1,0,0,0,0
3,3,8.004621,13.752827,67.942430,172.048517,18.357537,-1.905314,20.262851,9.505382,7.752109,...,59.217,1.203000,-18.36000,-69.5600,6.38,0,1,0,0,0
4,4,19.663313,6.021628,46.403303,214.393737,27.463324,14.379219,13.084105,21.727257,22.187209,...,0.000,0.004000,-18.40000,-70.3400,-1.04,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,135,15.688830,13.730183,58.347884,338.546730,28.680324,5.104891,23.575432,11.618111,18.571219,...,2.352,106.626000,-30.98022,-71.0612,4.16,0,0,1,0,0
134,136,15.272948,14.016008,59.113067,336.015721,28.228734,4.478181,23.750553,11.251113,18.414650,...,2.761,82.951999,-30.71162,-71.1770,5.09,0,0,1,0,0
135,137,15.272948,14.016008,59.113067,336.015721,28.228734,4.478181,23.750553,11.251113,18.414650,...,2.761,82.951999,-30.71162,-71.1770,6.32,0,0,1,0,0
136,138,15.211451,13.884745,59.017968,336.079665,28.098760,4.505688,23.593072,11.148939,18.697681,...,2.588,96.348999,-30.88676,-71.1931,3.06,0,0,1,0,0


In [88]:
norte, sur = [], []
for i in range(0, len(df['Latitude'])):
    
    if df['Atacama'][i]==1 or df['Arica'][i]==1:
        norte.append(1)
        sur.append(0)
        
    else:
        norte.append(0)
        sur.append(1)

In [89]:
df['Norte'] = norte
df['Sur'] = sur

df

,index,bio1,bio2,bio3,bio4,bio5,bio6,bio7,bio8,bio9,...,Latitude,Longitude,15N,Atacama,Arica,La_Serena,Santiago,Maule,Norte,Sur
0,0,16.680066,19.940985,73.603150,222.898539,30.221051,3.088484,27.132568,17.506520,18.646753,...,-22.85000,-69.2600,13.55,1,0,0,0,0,1,0
1,1,13.945570,16.422479,66.199236,302.465765,26.373077,1.530497,24.842580,13.844643,15.541803,...,-23.52000,-67.9700,2.92,1,0,0,0,0,1,0
2,2,17.292080,7.011714,49.380242,201.289604,25.103611,10.860042,14.243569,18.758515,19.791519,...,-23.50000,-70.5400,12.88,1,0,0,0,0,1,0
3,3,8.004621,13.752827,67.942430,172.048517,18.357537,-1.905314,20.262851,9.505382,7.752109,...,-18.36000,-69.5600,6.38,0,1,0,0,0,1,0
4,4,19.663313,6.021628,46.403303,214.393737,27.463324,14.379219,13.084105,21.727257,22.187209,...,-18.40000,-70.3400,-1.04,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,135,15.688830,13.730183,58.347884,338.546730,28.680324,5.104891,23.575432,11.618111,18.571219,...,-30.98022,-71.0612,4.16,0,0,1,0,0,0,1
134,136,15.272948,14.016008,59.113067,336.015721,28.228734,4.478181,23.750553,11.251113,18.414650,...,-30.71162,-71.1770,5.09,0,0,1,0,0,0,1
135,137,15.272948,14.016008,59.113067,336.015721,28.228734,4.478181,23.750553,11.251113,18.414650,...,-30.71162,-71.1770,6.32,0,0,1,0,0,0,1
136,138,15.211451,13.884745,59.017968,336.079665,28.098760,4.505688,23.593072,11.148939,18.697681,...,-30.88676,-71.1931,3.06,0,0,1,0,0,0,1


In [90]:
del df['index']

df

,bio1,bio2,bio3,bio4,bio5,bio6,bio7,bio8,bio9,bio10,...,Latitude,Longitude,15N,Atacama,Arica,La_Serena,Santiago,Maule,Norte,Sur
0,16.680066,19.940985,73.603150,222.898539,30.221051,3.088484,27.132568,17.506520,18.646753,19.269062,...,-22.85000,-69.2600,13.55,1,0,0,0,0,1,0
1,13.945570,16.422479,66.199236,302.465765,26.373077,1.530497,24.842580,13.844643,15.541803,17.465280,...,-23.52000,-67.9700,2.92,1,0,0,0,0,1,0
2,17.292080,7.011714,49.380242,201.289604,25.103611,10.860042,14.243569,18.758515,19.791519,20.085420,...,-23.50000,-70.5400,12.88,1,0,0,0,0,1,0
3,8.004621,13.752827,67.942430,172.048517,18.357537,-1.905314,20.262851,9.505382,7.752109,10.127124,...,-18.36000,-69.5600,6.38,0,1,0,0,0,1,0
4,19.663313,6.021628,46.403303,214.393737,27.463324,14.379219,13.084105,21.727257,22.187209,22.400360,...,-18.40000,-70.3400,-1.04,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,15.688830,13.730183,58.347884,338.546730,28.680324,5.104891,23.575432,11.618111,18.571219,20.023326,...,-30.98022,-71.0612,4.16,0,0,1,0,0,0,1
134,15.272948,14.016008,59.113067,336.015721,28.228734,4.478181,23.750553,11.251113,18.414650,19.505320,...,-30.71162,-71.1770,5.09,0,0,1,0,0,0,1
135,15.272948,14.016008,59.113067,336.015721,28.228734,4.478181,23.750553,11.251113,18.414650,19.505320,...,-30.71162,-71.1770,6.32,0,0,1,0,0,0,1
136,15.211451,13.884745,59.017968,336.079665,28.098760,4.505688,23.593072,11.148939,18.697681,19.452640,...,-30.88676,-71.1931,3.06,0,0,1,0,0,0,1


In [91]:
#df.to_csv('data/training_15N_suelos_v3.csv', index=False)

In [93]:
df_N_Soil = pd.read_csv('data/training_15N_suelos_v3.csv')
df_N_Plant = pd.read_csv('data/training_15N_plantas_v2.csv')
df_C_Soil = pd.read_csv('data/training_13C_suelos_v2.csv')
df_C_Plant = pd.read_csv('data/training_13C_plantas_v2.csv')

In [94]:
df_N_Soil = df_N_Soil.rename(columns={'15N': 'iso'})
df_N_Plant = df_N_Plant.rename(columns={'15N': 'iso'})
df_C_Soil = df_C_Soil.rename(columns={'13C': 'iso'})
df_C_Plant = df_C_Plant.rename(columns={'13C': 'iso'})

n=0
for i in [df_N_Soil, df_N_Plant, df_C_Soil, df_C_Plant]:
    n+=len(i['iso'])
n    

538

In [100]:
suelo = []
planta = []
N = []
C = []

for dfff, i in zip([df_N_Soil, df_N_Plant, df_C_Soil, df_C_Plant], [(1,0,1,0), (0,1,1,0), (1,0,0,1),(0,1,0,1)]):
    n = len(dfff['iso'])
    dfff['suelo'] = [i[0]]*n
    dfff['planta'] = [i[1]]*n
    dfff['N'] = [i[2]]*n
    dfff['C'] = [i[3]]*n

df = pd.concat([df_N_Soil, df_N_Plant, df_C_Soil, df_C_Plant])

df


,bio1,bio2,bio3,bio4,bio5,bio6,bio7,bio8,bio9,bio10,...,Arica,La_Serena,Santiago,Maule,Norte,Sur,suelo,planta,N,C
0,16.680066,19.940985,73.603150,222.898539,30.221051,3.088484,27.132568,17.506520,18.646753,19.269062,...,0,0,0,0,1,0,1,0,1,0
1,13.945570,16.422479,66.199236,302.465765,26.373077,1.530497,24.842580,13.844643,15.541803,17.465280,...,0,0,0,0,1,0,1,0,1,0
2,17.292080,7.011714,49.380242,201.289604,25.103611,10.860042,14.243569,18.758515,19.791519,20.085420,...,0,0,0,0,1,0,1,0,1,0
3,8.004621,13.752827,67.942430,172.048517,18.357537,-1.905314,20.262851,9.505382,7.752109,10.127124,...,1,0,0,0,1,0,1,0,1,0
4,19.663313,6.021628,46.403303,214.393737,27.463324,14.379219,13.084105,21.727257,22.187209,22.400360,...,0,0,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,14.121252,13.251624,56.089936,355.066326,26.990337,3.313603,23.676734,9.829689,17.174319,18.535722,...,0,1,0,0,0,1,0,1,0,1
123,14.781758,14.284080,57.344513,352.678578,28.363834,3.405418,24.958416,10.496199,18.252404,19.212604,...,0,1,0,0,0,1,0,1,0,1
124,14.781758,14.284080,57.344513,352.678578,28.363834,3.405418,24.958416,10.496199,18.252404,19.212604,...,0,1,0,0,0,1,0,1,0,1
125,14.781758,14.284080,57.344513,352.678578,28.363834,3.405418,24.958416,10.496199,18.252404,19.212604,...,0,1,0,0,0,1,0,1,0,1


In [101]:
df.to_csv('data/training_All_v1.csv', index=False)